Подключение библиотек:

In [ ]:
import glob
from PIL import Image

import torch

#using numpy
import numpy as np

#for data load or save
import pandas as pd

#visualize some datasets
import matplotlib.pyplot as plt

#check our work directory
import os